In [1]:
import pandas as pd
import os
import json
import numpy as np
from functions import Convertor_Single
from functions import Convertor_Multiple
from functions import df_info
from functions import frame_num_drop
from functions import df_reshape
from functions import random_forest_classifier
from functions import df_pivot_multiple
from functions import pivot_diff_lag
from functions import merge_json_data
from functions import retrieve_and_merge_data
from functions import num_to_cik
from functions import pivot_lag
from functions import df_asset_target
from functions import process_in_chunks
from functions import df_target_dropna
from functions import train_random_forest
from functions import save_model
from functions import load_model
from functions import evaluate_model
from functions import everything

## STEP 01: Convert & Merge:

In [2]:
df = pd.read_csv('Clustering Structure/df_clustered_80.csv')

In [3]:
len(df['CIK'].unique())

51

In [4]:
df_list = list(df['CIK'].unique())

In [5]:
df_new_list = num_to_cik(df_list)

In [6]:
merged_df = Convertor_Multiple(df_new_list)

In [7]:
df_info(merged_df)

(0.96, (3543, 3277))

## STEP 02: Frames & Numeric & Drop Missing EPSmerged_df

In [2]:
merged_df = pd.read_csv('Clustering Structure/df_clustered_80.csv')

In [3]:
df_num = frame_num_drop(merged_df)

/Users/khatansanaabandi/Documents/Python/SEC V3/functions.py:194: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  df = pd.pivot_table(df, index=['CIK', 'Frame'])


In [4]:
df_info(df_num)

(0.0, (1944, 141))

## STEP 03: Divide Asset, Target, Lag

### 3.1 Divide by Asset & Target

In [5]:
divided = df_asset_target(df_num)

In [6]:
divided

,CIK,Frame,AccountsPayableCurrent,AccountsReceivableNetCurrent,AccruedIncomeTaxesCurrent,AccruedLiabilitiesCurrent,AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment,AccumulatedOtherComprehensiveIncomeLossDefinedBenefitPensionAndOtherPostretirementPlansNetOfTax,AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax,AccumulatedOtherComprehensiveIncomeLossNetOfTax,...,StockholdersEquity,StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest,TreasuryStockShares,TreasuryStockValue,UnrecognizedTaxBenefits,UnrecognizedTaxBenefitsIncomeTaxPenaltiesAndInterestAccrued,UnrecognizedTaxBenefitsThatWouldImpactEffectiveTaxRate,WeightedAverageNumberDilutedSharesOutstandingAdjustment,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic
0,89439,CY2012Q1,0.067662,0.218087,0.000000,0.0,0.000000,0.000000,0.000000,-0.028907,...,0.617627,0.638134,0.0,0.030878,0.002697,0.000207,0.000761,0.000307,0.026597,0.026290
1,89439,CY2012Q2,0.070663,0.235189,0.000000,0.0,0.000000,0.000000,0.000000,-0.036112,...,0.710716,0.734374,0.0,0.034938,0.003065,0.000236,0.000864,0.000343,0.030229,0.029887
2,89439,CY2012Q3,0.063029,0.251117,0.000000,0.0,0.000000,0.000000,0.000000,-0.038444,...,0.476274,0.505197,0.0,0.452012,0.001251,0.000096,0.000385,0.000435,0.036518,0.036083
3,89439,CY2012Q4,0.079313,0.245521,0.000000,0.0,0.441622,0.000012,-0.002746,-0.038602,...,0.459091,0.487220,0.0,0.424282,0.002952,0.000181,0.000000,0.000000,0.000000,0.000000
4,89439,CY2013Q1,0.071869,0.272638,0.000000,0.0,0.000000,0.000000,0.000000,-0.042048,...,0.459529,0.486878,0.0,0.409566,0.003062,0.000000,0.000612,0.000654,0.049341,0.048687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2170,310764,CY2022Q3,0.000000,0.086235,0.006753,0.0,0.076036,0.000000,0.000000,-0.002473,...,0.000000,0.457522,0.0,0.000000,0.000000,0.000000,0.000000,0.000094,0.010611,0.010516
2171,310764,CY2022Q4,0.000000,0.096654,0.007917,0.0,0.076863,0.000000,0.000000,-0.005992,...,0.000000,0.450493,0.0,0.000000,0.007754,0.000000,0.000000,0.000000,0.000000,0.000000
2172,310764,CY2023Q1,0.000000,0.087293,0.010073,0.0,0.079636,0.000000,0.000000,-0.008281,...,0.000000,0.458729,0.0,0.000000,0.000000,0.000000,0.000000,0.000114,0.010405,0.010291
2173,310764,CY2023Q2,0.000000,0.087172,0.007966,0.0,0.080328,0.000000,0.000000,-0.008821,...,0.000000,0.464086,0.0,0.000000,0.000000,0.000000,0.000000,0.000112,0.010262,0.010150


### 3.2 Lagged

In [ ]:
merged = process_in_chunks(divided,'CIK','Frame',5,5)

In [9]:
df_info(merged)

(0.07, (1944, 828))

In [10]:
merged.to_csv('3.3 pivot_divide_asset.csv',mode='w',index=True)

### 3.3 target

In [11]:
df = pd.read_csv('3.3 pivot_divide_asset.csv')

In [20]:
df_targeted=df_target_dropna(merged,'CIK','Frame','EarningsPerShareBasic_5')

In [13]:
df_targeted.to_csv('3.3.3 targeted.csv',mode='w',index=False)

## STEP 04: Shuffle the rows and split into train and validation.

In [14]:
df = pd.read_csv('3.3.3 targeted.csv')

In [15]:
df.shape

(1689, 829)

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming 'df' is your DataFrame
# Shuffle the dataset
df_shuffled = df.sample(frac=1, random_state=42)

# Split the dataset into a 70/30 ratio
train_set, test_set = train_test_split(df_shuffled, test_size=0.3, random_state=42)

# Save the datasets into different CSV files
train_set.to_csv('train_set.csv',mode='w', index=False)
test_set.to_csv('test_set.csv',mode='w',index=False)


## STEP 05: Train, save, load, predict & compare

### 5.1 Model on train

In [20]:
df = pd.read_csv('train_set.csv')

In [21]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from joblib import dump, load

# Assuming 'df' is your first part of the dataset
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('target', axis=1), df['target'], test_size=0.3, random_state=42
)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate Accuracy and AUC/ROC
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, rf_classifier.predict_proba(X_test)[:, 1])

print(f"Accuracy: {accuracy}")
print(f"AUC/ROC: {roc_auc}")

KeyError: "['real_target'] not found in axis"

### 5.2 Save the Model

In [6]:
# Save the model
dump(rf_classifier, 'random_forest_classifier.joblib')


['random_forest_classifier.joblib']

### 5.3 Load the Model and Run on Unseen Data

In [14]:
df2 = pd.read_csv('test_set.csv')

In [15]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from joblib import dump, load
# Load the model in a new session
rf_classifier_loaded = load('random_forest_classifier.joblib')

In [16]:
# Assuming 'df2' is your second part of the dataset
# Predict on the second part of the dataset
y_pred_unseen = rf_classifier_loaded.predict(df2.drop('real_target', axis=1))


In [17]:
# Calculate Accuracy and AUC/ROC for the unseen dataset
accuracy_unseen = accuracy_score(df2['real_target'], y_pred_unseen)
roc_auc_unseen = roc_auc_score(df2['real_target'], rf_classifier_loaded.predict_proba(df2.drop('real_target', axis=1))[:, 1])

print(f"Accuracy on unseen data: {accuracy_unseen}")
print(f"AUC/ROC on unseen data: {roc_auc_unseen}")

Accuracy on unseen data: 0.6134122287968442
AUC/ROC on unseen data: 0.6332902603063894


### 5.4 Function

In [18]:
df = pd.read_csv('train_set.csv')
df2 = pd.read_csv('test_set.csv')

In [19]:
rf_model, train_accuracy, train_roc_auc = train_random_forest(df, 'target')
print(f"Training Accuracy: {train_accuracy}")
print(f"Training AUC/ROC: {train_roc_auc}")

Training Accuracy: 0.6253521126760564
Training AUC/ROC: 0.6796056391693656


In [26]:
save_model(rf_model, 'random_forest_classifier.joblib')

In [3]:
rf_model_loaded = load_model('random_forest_classifier.joblib')

In [4]:
unseen_accuracy, unseen_roc_auc = evaluate_model(rf_model_loaded, df2, 'target')
print(f"Accuracy on unseen data: {unseen_accuracy}")
print(f"AUC/ROC on unseen data: {unseen_roc_auc}")

Accuracy on unseen data: 0.579225352112676
AUC/ROC on unseen data: 0.626072607260726


## STEP 06: Test Hypothesis

### 6.1 Top 200 columns

In [4]:
df = pd.read_csv('Clustering Structure/df_clustered_80.csv')
df_list = list(df['CIK'].unique())
df_new_list = num_to_cik(df_list)
merged_df = Convertor_Multiple(df_new_list)
df_num = frame_num_drop(merged_df)
divided = df_asset_target(df_num)
merged = process_in_chunks(divided,'CIK','Frame',0,5)
df_targeted=df_target_dropna(merged,'target_lag5')
df_targeted = df_targeted.drop(['CIK','Frame'], axis=1)
shuffled_df = df_targeted.sample(frac=1)#.reset_index(drop=True)
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming 'df' is your DataFrame
# Shuffle the dataset
df_shuffled = df_targeted.sample(frac=1, random_state=42)

# Split the dataset into a 70/30 ratio
train_set, test_set = train_test_split(df_shuffled, test_size=0.3, random_state=42)

# Save the datasets into different CSV files
train_set.to_csv('train_set.csv',mode='w', index=False)
test_set.to_csv('test_set.csv',mode='w',index=False)

/Users/khatansanaabandi/Documents/Python/SEC V3/functions.py:194: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  df = pd.pivot_table(df, index=['CIK', 'Frame'])


In [4]:
df = pd.read_csv('train_set.csv')
df2 = pd.read_csv('test_set.csv')
rf_model, train_accuracy, train_roc_auc = train_random_forest(df, 'real_target')
print(f"Training Accuracy: {train_accuracy}")
print(f"Training AUC/ROC: {train_roc_auc}")
save_model(rf_model, 'random_forest_classifier.joblib')

Training Accuracy: 0.9975490196078431
Training AUC/ROC: 0.999813321385902


In [ ]:
rf_model_loaded = load_model('random_forest_classifier.joblib')
unseen_accuracy, unseen_roc_auc = evaluate_model(rf_model_loaded, df2, 'real_target')
print(f"Accuracy on unseen data: {unseen_accuracy}")
print(f"AUC/ROC on unseen data: {unseen_roc_auc}")

### 6.2 Different Lags

## Test Again

In [15]:
df=df_target_dropna(merged,'CIK','Frame','EarningsPerShareBasic_5')

In [16]:
df

,AcceleratedShareRepurchasesFinalPricePaidPerShare,AcceleratedShareRepurchasesFinalPricePaidPerShare_1,AcceleratedShareRepurchasesFinalPricePaidPerShare_2,AcceleratedShareRepurchasesFinalPricePaidPerShare_3,AcceleratedShareRepurchasesFinalPricePaidPerShare_4,AcceleratedShareRepurchasesFinalPricePaidPerShare_5,AcceleratedShareRepurchasesSettlementPaymentOrReceipt,AcceleratedShareRepurchasesSettlementPaymentOrReceipt_1,AcceleratedShareRepurchasesSettlementPaymentOrReceipt_2,AcceleratedShareRepurchasesSettlementPaymentOrReceipt_3,...,WorkersCompensationLiabilityCurrent_3,WorkersCompensationLiabilityCurrent_4,WorkersCompensationLiabilityCurrent_5,WriteOffOfDeferredDebtIssuanceCost,WriteOffOfDeferredDebtIssuanceCost_1,WriteOffOfDeferredDebtIssuanceCost_2,WriteOffOfDeferredDebtIssuanceCost_3,WriteOffOfDeferredDebtIssuanceCost_4,WriteOffOfDeferredDebtIssuanceCost_5,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
